The HoloPlot plotting interface supports streamz DataFrame and Series objects generating streaming plots. To support plotting streaming data you can use this interface either in a Jupyter notebook or deploy it as a bokeh server app.

HoloViews provides several constructs which make it well suited to streaming visualizations. All plotting methods will return so called ``DynamicMap`` objects, which update the plot whenever streamz triggers an event. For additional information about working and plotting with HoloViews see the [User Guide](http://holoviews.org/user_guide/index.html), as we will focus on using the high-level plotting API in this overview and skip most of the mechanics going on behind the scenes.

*All plots generated by the streamz plotting interface dynamically stream data, since the documentation cannot easily embed streaming plots all plots represent static screenshots.*

As in the previous sections we start by patching the streamz library with the plotting API:

In [ ]:
import holoviews as hv
import holoplot as hvplot
hvplot.patch('streamz')
hv.extension('bokeh')

## Basic plotting

Throughout this section we will be using the ``Random`` construct, which provides an easy way of generating a DataFrame of random streaming data.

In [ ]:
from streamz.dataframe import Random
df = Random()

<img src='images/random_df.png' width=300px></img>

The plot method on Series and DataFrame is a simple wrapper around a
line plot, which will plot all columns:

In [ ]:
df.plot()

<center><img src='images/simple_plot.png' width=700px></img></center>

The plot method can also be called on a Series, plotting a specific column:

Another more general way to express the same thing is to explicitly define ``x`` and ``y`` in the DataFrame plot method:

In [ ]:
df.z.cumsum().plot()

<center><img src='images/series_plot.png' width=700px></img></center>

## Other plots

Plotting methods allow for a handful of plot styles other than the
default Line plot. These methods can be provided as the ``kind``
keyword argument to :meth:`~DataFrame.plot`.
These include:

* 'bar' or 'barh' <plotting.barplot> for bar plots
* 'histogram' for histogram
* 'box' for boxplot
* 'kde' or 'density' for density plots
* 'area' for area plots
* 'scatter' for scatter plots
* 'table' for tables

For example, a bar plot can be created the following way:

In [ ]:
df.groupby('y').x.sum().plot('y', 'x', kind='bar')

<center><img src='images/bar_plot.png' width=700px></img></center>

You can also create these other plots using the methods ``DataFrame.plot.<kind>`` instead of providing the ``kind`` keyword argument. This makes it easier to discover plot methods and the specific arguments they use:

### Bar plots

For labeled, non-time series data, you may wish to produce a bar plot.
In addition to the simple bar plot shown above we can also produce
grouped bars:

In [ ]:
df.groupby('y').sum().plot.bar(x='y')

<center><img src='images/bar_grouped.png' width=700px></img></center>

Alternatively you may also stack the bars:

In [ ]:
#df.groupby('y').sum().plot.bar(x='y', stacked=True)

<center><img src='images/bar_stacked.png' width=700px></img></center>

## Area plots

You can create area plots with Series.plot.area() and DataFrame.plot.area(). To produce stacked area plot, each column must be either all positive or all negative values.


In [ ]:
df.x.plot.area()

<center><img src='images/series_area.png' width=700px></img></center>

When plotting multiple columns on a DataFrame the areas may be stacked:

In [ ]:
df[['x', 'y']].plot.area(stacked=True)

<center><img src='images/stacked_area.png' width=700px></img></center>

## Scatter plots

Scatter plot can be drawn by using the DataFrame.plot.scatter() method. Scatter plot requires numeric or datetime columns for x and y axis. These can be specified by x and y keywords each.

In [ ]:
df.plot.scatter(x='x', y='z')

<center><img src='images/df_scatter.png' width=350px></img></center>

The scatter points can also be colored by a certain column using the c keyword. Additionally we will enable a colorbar and adjust the xaxis by setting x-axis limits using xlim:


In [ ]:
df.plot.scatter(x='y', y='z', c='x', cmap='viridis', width=400, colorbar=True, xlim=(-1, 6))

<center><img src='images/colored_scatter.png' width=450px></img></center>

### Histograms

Histogram can be drawn by using the DataFrame.plot.hist() and Series.plot.hist() methods. The number of bins can be declared using
the bins keyword and normalization can be disabled with the normed keyword.

In [ ]:
df.z.plot.hist('z', bins=50, backlog=5000, normed=False)

In [ ]:
df.plot.hist(bin_range=(-3, 3), bins=50, backlog=5000, alpha=0.3)

### Box Plots

Boxplot can be drawn calling Series.plot.box() and DataFrame.plot.box() to visualize the distribution of values within each column.


In [ ]:
df.plot.box()

<center><img src='images/df_boxplot.png' width=700px></img></center>

Or we can generate a boxplot of a Series:

In [ ]:
df.x.plot.box(width=400)

<center><img src='images/series_boxplot.png' width=350px></img></center>

It is also possible to group a box plot by a secondary variable:


In [ ]:
df.plot.box('x', by='y', height=400)

<center><img src='images/grouped_boxplot.png' width=700px></img></center>

### KDE plots

You can create density plots using the Series.plot.kde() and DataFrame.plot.kde() methods.

<center><img src='images/series_kde.png' width=700px></img></center>

### Tables

We can also stream a table view of the data:

In [ ]:
df.plot.table(width=600)

<center><img src='images/df_table.png' width=600px></img></center>

## Composing Plots

One of the core strengths of HoloViews is the ease of composing different plots. Individual plots can be composed using the ``*`` and ``+`` operators, which overlay and compose plots into layouts respectively. For more information on composing objects see the HoloViews [User Guide](https://holoviews.org/user_guide/Composing_Elements.html)

By using these operators we can combine multiple plots into composite Overlay and Layout objects, and lay them out in two columns using the ``Layout.cols`` method:

In [ ]:
(df.plot.line(width=400) * df.plot.scatter(width=400) +
 df.groupby('y').sum().plot.bar('y', 'x', width=400) +
 df.plot.box(width=400) + df.x.plot.kde(width=400)).cols(2)

<center><img src='images/composite_plot.png' width=700px></img></center>

## Deployment as bokeh apps

In the Jupyter notebook HoloViews objects will automatically be rendered, but when deploying a plot as a bokeh app it has to be rendered explicitly.

The following examples describes how to set up a streaming DataFrame, declare some plots, compose them, set up a callback to update the plot
and finally convert the composite plot to a bokeh Document, which can be served from a script using ``bokeh serve`` on the commandline.

```python

import numpy as np
import pandas as pd
import holoviews as hv
from streamz import Stream
from streamz.dataframe import DataFrame
import hvplot

hvplot.patch('streamz')

renderer = hv.renderer('bokeh').instance(mode='server')

# Set up streaming DataFrame
stream = Stream()
index = pd.DatetimeIndex([])
example = pd.DataFrame({'x': [], 'y': [], 'z': []},
                       columns=['x', 'y', 'z'], index=)
df = DataFrame(stream, example=example)
cumulative = df.cumsum()[['x', 'z']]

# Declare plots
line = cumulative.plot.line(width=400)
scatter = cumulative.plot.scatter(width=400)
bars = df.groupby('y').sum().plot.bar(width=400)
box = df.plot.box(width=400)
kde = df.x.plot.kde(width=400)

# Compose plots
layout = (line * scatter + bars + box + kde).cols(2)

# Set up callback with streaming data
def emit():
    now = pd.datetime.now()
    delta = np.timedelta64(500, 'ms')
    index = pd.date_range(np.datetime64(now)-delta, now, freq='100ms')
    df = pd.DataFrame({'x': np.random.randn(len(index)),
                       'y': np.random.randint(0, 10, len(index)),
                       'z': np.random.randn(len(index))},
                      columns=['x', 'y', 'z'], index=index)
    stream.emit(df)

# Render layout to bokeh server Document and attach callback
doc = renderer.server_doc(layout)
doc.title = 'Streamz HoloViews based Plotting API Bokeh App Demo'
doc.add_periodic_callback(emit, 500)
```
<center><img src='images/server_plot.png' width=600px></img></center>

For more details on deploying bokeh apps see the HoloViews [User Guide](https://holoviews.org/user_guide/Deploying_Bokeh_Apps.html).

## Using HoloViews directly

HoloViews includes first class support for streamz DataFrame and Series, for more details see the [Streaming Data section](https://holoviews.org/user_guide/Streaming_Data.html) in the
HoloViews documentation.